In [45]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
import random
import os

In [46]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Initialize lists
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Load intents file
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [47]:
# Obtener el directorio actual del proyecto
project_path = os.getcwd()

# Crear subdirectorio nltk_data si no existe
nltk_data_path = os.path.join(project_path, 'nltk_data')
os.makedirs(nltk_data_path, exist_ok=True)

# Configurar la ruta de descarga de NLTK
nltk.download('punkt_tab', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)

# Añadir la nueva ruta a las rutas de búsqueda de NLTK
nltk.data.path.append(nltk_data_path)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     f:\IABD\Modelos\UD03\chatbot\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     f:\IABD\Modelos\UD03\chatbot\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
# Procesar las intenciones
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenizar cada palabra
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # Agregar documentos al corpus
        documents.append((w, intent['tag']))
        
        # Agregar a la lista de clases
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [49]:
# Lemmatize, lower each word, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Documents = combination between patterns and intents
print(len(documents), "documents")

# Classes = intents
print(len(classes), "classes", classes)

# Words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

# Save words and classes to pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

2 documents
1 classes ['greeting']
3 unique lemmatized words ['estã¡s', 'hola', 'â¿cã³mo']


In [50]:
# Create our training data
training = []

# Create an empty array for our output
output_empty = [0] * len(classes)

# Get the maximum length of words
max_words_length = len(words)

# Training set: bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []

    # List of tokenized words for the pattern
    pattern_words = doc[0]

    # Lemmatize each word to create a base word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Create our bag of words array with 1 if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Ensure all bags have the same length
    bag = bag + [0] * (max_words_length - len(bag))

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Append the bag of words and the output_row to training data
    training.append([bag, output_row])

# Shuffle our features and convert into np.array
random.shuffle(training)

# Convert training data to numpy arrays
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

print("Training data created")


Training data created


In [51]:
# Create model - 3 layers.
# First layer: 128 neurons, 
# Second layer: 64 neurons, 
# Third output layer: number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
# Using stochastic gradient descent with Nesterov accelerated gradient for good results
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# EarlyStopping callback to stop training if the validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, validation_split=0.1, callbacks=[early_stopping])

# Save the model after training
model.save('chatbot_model.h5')

print("Model created")

Epoch 1/200


c:\Python312\Lib\site-packages\keras\src\ops\nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Python312\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 814ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


Model created
